In [141]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [142]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }

# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
param = {
    'year': year,
    'month': month,
    'day': day
}
# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '13'
# }


url_tour = 'tours/'
# url_tour = 'tours/34256'
# url_tour = 'tours/34279'

session = requests.Session()

In [143]:
# делаем get-запрос
response = session.get(url+url_tour, params=param, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


In [144]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

In [145]:
tours = []
for row, item in zip(table[1:], ref):
    res = session.get(url+item, headers=header)
    tour = html.fromstring(res.text)

    player1 = tour.xpath('.//td[@class="right"]/a/text()')
    player2 = tour.xpath('.//td[@class="left"]/a/text()')
    score = tour.xpath('.//td[@class="score"]//a/text()')
    url_match = tour.xpath('.//td[@class="score"]//a/@href')
    if len(tour.xpath('//table')) > 3:
        score_set = tour.xpath('.//small[@class="nowrap"]/text()')
        for i in range(len(score)):
            if score[i] == "- : -":
                score_set.insert(i, 'не сыгран')
    else: score_set = ['не сыгран' for _ in range(len(score))]
        

    tours_info = {
        'player1': player1,
        'player2': player2,
        'score': score ,
        'url_match': url_match,
        'score_set': score_set
    }

    tours.append(tours_info)

In [146]:
for el in tours:
    print(el)

{'player1': ['Руденко В', 'Беспалов Н', 'Михайлов И', 'Руденко В', 'Федоров А', 'Михайлов И', 'Лбов Ю', 'Беспалов Н', 'Лбов Ю', 'Федоров А'], 'player2': ['Михайлов И', 'Лбов Ю', 'Федоров А', 'Беспалов Н', 'Лбов Ю', 'Беспалов Н', 'Руденко В', 'Федоров А', 'Михайлов И', 'Руденко В'], 'score': ['- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -'], 'url_match': ['games/322752', 'games/322753', 'games/322754', 'games/322755', 'games/322756', 'games/322757', 'games/322758', 'games/322759', 'games/322760', 'games/322761'], 'score_set': ['не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран']}
{'player1': ['Вашков А', 'Иванов А-р', 'Кустов Д', 'Вашков А', 'Кутузов О', 'Кустов Д', 'Денисов А-й', 'Иванов А-р', 'Денисов А-й', 'Кутузов О'], 'player2': ['Кустов Д', 'Денисов А-й', 'Кутузов О', 'Иванов А-р', 'Денисов А-й', 'Иванов А-р', 'Вашков А', 'Кутузов О', 'Кустов Д', 'Вашков А'], 'scor

In [147]:
df = pd.DataFrame(columns=['player1', 'player2', 'score', 'url_match', 'score_set'])
for el in tours:
    df_el = pd.DataFrame(el)
    df = df.append(df_el, ignore_index=True)
df['url_match'] = url + df['url_match']

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6496\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6496\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6496\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6496\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Loca

In [150]:
df

,player1,player2,score,url_match,score_set
0,Руденко В,Михайлов И,- : -,https://tt.sport-liga.pro/games/322752,не сыгран
1,Беспалов Н,Лбов Ю,- : -,https://tt.sport-liga.pro/games/322753,не сыгран
2,Михайлов И,Федоров А,- : -,https://tt.sport-liga.pro/games/322754,не сыгран
3,Руденко В,Беспалов Н,- : -,https://tt.sport-liga.pro/games/322755,не сыгран
4,Федоров А,Лбов Ю,- : -,https://tt.sport-liga.pro/games/322756,не сыгран
...,...,...,...,...,...
187,Иванов А-й,Чернявский М,2 : 3,https://tt.sport-liga.pro/games/322697,(12-10 20-18 7-11 5-11 9-11)
188,Чернявский М,Ланин С,3 : 1,https://tt.sport-liga.pro/games/322698,(11-5 6-11 11-7 14-12)
189,Кольмин А,Иванов А-й,2 : 3,https://tt.sport-liga.pro/games/322699,(11-8 5-11 8-11 11-5 7-11)
190,Чернявский М,Ланин С,3 : 0,https://tt.sport-liga.pro/games/322982,(11-9 11-4 11-7)


In [149]:
# df.to_csv(f'tours_{date}.csv', index=False)